In [140]:
import pandas as pd
from pptx import Presentation
from pptx.util import Inches
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from pptx.oxml import parse_xml
from pptx.oxml.ns import nsdecls
from pptx.util import Pt
import sys
from pptx.enum.shapes import MSO_CONNECTOR
import numpy as np

In [141]:
def build_hierarchy(df, employee_id, level=1):
    """
    Recursively finds all employees under a given employee ID and assigns a level to them.
    
    Args:
        df (pd.DataFrame): The dataframe containing 'Employee ID' and 'Supervisor ID'.
        employee_id (int or str): The employee ID from which to start building the hierarchy.
        level (int): The current hierarchical level (default is 1 for the given employee_id).

    Returns:
        pd.DataFrame: A dataframe with all original columns plus 'Level', containing only employees under the given employee ID.
    """
    hierarchy = []
    root = df[df['Employee ID'] == employee_id]
    root = root.reset_index(drop=True)
    
    def recurse(emp_id, lvl):
        
        subordinates = df[df['Supervisor ID'] == emp_id]
        for _, row in subordinates.iterrows():
            row_dict = row.to_dict()
            row_dict['Level'] = lvl
            hierarchy.append(row_dict)
            recurse(row['Employee ID'], lvl + 1)
    
    # Check if the given employee exists in the dataset
    if employee_id not in df['Employee ID'].values:
        return pd.DataFrame(columns=df.columns.tolist() + ['Level'])
    
    # Start with the given employee
    root_employee = df[df['Employee ID'] == employee_id].iloc[0].to_dict()
    root_employee['Level'] = level
    hierarchy.append(root_employee)
    
    recurse(employee_id, level + 1)
    
    return pd.DataFrame(hierarchy), root


def group(result_df):
    grouped = result_df.groupby(["Supervisor ID", "Level"])

    two_level_heirarchy_dict = {}

    for _, group in grouped:
        
        if group.iloc[0]['Level'] != 1: # if not root
            
            two_level_heirarchy = []
            supervisor_id = int(group.iloc[0]["Supervisor ID"])
            
            for _, row in group.iterrows():
                two_level_heirarchy.append(row.to_dict())
            #two_level_heirarchy.append(False) # To indicate if the structure has been plotted
                
                
            two_level_heirarchy_dict[int(supervisor_id)] = two_level_heirarchy
            
    return two_level_heirarchy_dict

In [146]:
class Chart:
    
    def __init__(self, root, second_level_root, remaining_branch, coordinates):
        self.root = root
        self.second_level_root = second_level_root # list with dictionaries
        self.remaining_branch = remaining_branch # # list with dictionaries
        self.coordinates = coordinates 
        
    
    def add_rectangle_root(self, slide, left, top, width, height):
        """Adds a rectangle with text at the specified position."""
        
        '''
        left -	X-coordinate (distance from the left side of the slide) 
        top	 -  Y-coordinate (distance from the top of the slide) 
        width -	Width of the shape 
        height -  Height of the shape 
        '''
        
        text = f"{self.root['Department']}\n{self.root['Name']} ({self.root['Employee ID']})"
        
        shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, left, top, width, height)
        shape.fill.solid()
        shape.fill.fore_color.rgb = RGBColor(10, 80, 110)  # Dark blue
        shape.text_frame.text = text
        shape.text_frame.paragraphs[0].font.size = Pt(16)
        shape.text_frame.paragraphs[1].font.size = Pt(16)
        shape.text_frame.paragraphs[0].font.bold = True
        shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)  # White text
        self.coordinates[self.root['Employee ID']] = shape
        
        
    def plot_middle(self, slide):
        
        root_coor = self.coordinates.pop(next(iter(self.coordinates)))
        level2_top = root_coor.top + root_coor.height + Inches(0.5)  # Some space below the root
        root_bottom = root_coor.top + root_coor.height
        middle_x = root_coor.left + root_coor.width / 2
        middle_y = (root_coor.top + root_coor.height + level2_top) / 2
        
        # Vertical
        slide.shapes.add_connector(MSO_CONNECTOR.STRAIGHT, int(middle_x), int(root_bottom), int(middle_x), int(middle_y))
        
    
    def plot_second_level_branch(self, slide):
        num_branch = len(self.second_level_root)
        middle_y = (2 * Inches(1) + 2 * Inches(0.8) + Inches(0.5)) / 2
        lower_y = middle_y + Inches(0.28) 
        curr_right_most = 0
        
        for n in range(num_branch):
            if n != 0:
                curr_right_most = curr_right_most + Inches(0.5)
            
            employee_coor = {}
            
            # Connect level 1 and 2
            slide.shapes.add_connector(
                MSO_CONNECTOR.STRAIGHT, 
                int(curr_right_most + Inches(0.5)), 
                int(middle_y), 
                int(curr_right_most + Inches(0.5)), 
                int(lower_y)
            )
            
            curr_level2_root = self.second_level_root[n]
            curr_branch = self.remaining_branch[n]
            curr_level2_id = curr_level2_root['Employee ID']
            
            text = f"{curr_level2_root['Department']}\n{curr_level2_root['Name']} ({curr_level2_id})"
            
            # Plot level 2 root
            shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, curr_right_most, lower_y, Inches(1), Inches(0.6))
            shape.fill.solid()
            shape.fill.fore_color.rgb = RGBColor(10, 80, 110)  # Dark blue
            shape.text_frame.text = text
            shape.text_frame.paragraphs[0].font.size = Pt(12)
            shape.text_frame.paragraphs[1].font.size = Pt(12)
            shape.text_frame.paragraphs[0].font.bold = True
            shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)  # White text
                
            employee_coor[curr_level2_id] = shape
            
            if curr_level2_id in curr_branch:
                offset_x = shape.left + Inches(0.1)
                x_offset_h = shape.width / 4
                current_bottom = lower_y + Inches(0.6)
                
                # Store all y-positions for perfect vertical line sizing
                y_positions = []
                
                for i in range(len(curr_branch[curr_level2_id])):
                    # Calculate y-position with gap
                    y_pos = current_bottom
                    gap = Inches(0.5)
                    
                    subordinate = curr_branch[curr_level2_id][i]
                    y_positions.append(y_pos + gap)  # Store connection point
                    
                    # Horizontal line
                    slide.shapes.add_connector(
                        MSO_CONNECTOR.STRAIGHT,
                        int(offset_x), int(y_pos + gap),
                        int(offset_x + x_offset_h), int(y_pos + gap)
                    )

                    # Add rectangle
                    shape_v2 = slide.shapes.add_shape(
                        MSO_SHAPE.RECTANGLE,
                        int(offset_x + x_offset_h),
                        int(y_pos + gap / 2 + Inches(0.05)),
                        Inches(0.8),
                        Inches(0.4)   
                    )
                    shape_v2.fill.solid()
                    shape_v2.fill.fore_color.rgb = RGBColor(10, 80, 110)
                    text = f"{subordinate['Department']}\n{subordinate['Name']} ({subordinate['Employee ID']})"
                    shape_v2.text_frame.text = text
                    shape_v2.text_frame.paragraphs[0].font.size = Pt(10)
                    shape_v2.text_frame.paragraphs[1].font.size = Pt(10)
                    shape_v2.text_frame.paragraphs[0].font.bold = True
                    shape_v2.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
                    
                    current_bottom = shape_v2.top + shape_v2.height + Inches(0.3)
                    
                    # Process deeper levels
                    _, right_most, bottom = self.deep_search(
                        slide, 
                        {"target": (subordinate['Employee ID'], shape_v2)}, 
                        curr_branch
                    )
                    
                    if right_most > curr_right_most:
                        curr_right_most = right_most
                    if bottom > current_bottom:
                        current_bottom = bottom
                
                # Draw perfect vertical line after knowing all positions
                if y_positions:
                    min_y = min(y_positions) - Inches(0.5)  # Start from top connector
                    max_y = max(y_positions)  # End at last connector
                    slide.shapes.add_connector(
                        MSO_CONNECTOR.STRAIGHT,
                        int(offset_x), int(min_y),
                        int(offset_x), int(max_y)
                    )
                    

    def deep_search(self, slide, target, branch, right_most_coor=0, bottom_most_coor=0):
        target_id = target['target'][0]
        target_shape = target['target'][1]
        offset_x = target_shape.left + Inches(0.1)
        x_offset_h = target_shape.width / 4
        gap = 0.95 * target_shape.height  
            
        if target_id in branch:
            subordinates = branch[target_id]
            total_height_used = 0  # Tracks cumulative height used by this branch
            node_positions = []  # Store all node positions in this level
            
            for i, subordinate in enumerate(subordinates):
                # Calculate y-position based on previous subordinates' heights
            
                
                y_pos = target_shape.top + (1 + 0.95 * (i + total_height_used)) * target_shape.height
                
                # Draw the subordinate box
                text = f"{subordinate['Department']}\n{subordinate['Name']} ({subordinate['Employee ID']})"
                
                # Store node position for main vertical line
                node_positions.append(y_pos + gap/2)  # Middle of the horizontal connector
                
                # Horizontal line (connecting to the box)
                slide.shapes.add_connector(MSO_CONNECTOR.STRAIGHT, int(offset_x), int(y_pos + gap), int(offset_x + x_offset_h), int(y_pos + gap))
                
                # Subordinate rectangle
                shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, int(offset_x + x_offset_h), int(y_pos + gap / 2 + Inches(0.05)), Inches(0.8), Inches(0.4))
                
                # Update right-most coordinate if current shape extends further
                current_right = offset_x + x_offset_h + Inches(0.8)
                if current_right > right_most_coor:
                    right_most_coor = current_right
                
                # Update bottom-most coordinate if current shape extends lower
                current_bottom = y_pos + gap / 2 + Inches(0.05) + Inches(0.4)
                if current_bottom > bottom_most_coor:
                    bottom_most_coor = current_bottom
                
                shape.text_frame.text = text
                shape.text_frame.paragraphs[0].font.size = Pt(10)
                shape.text_frame.paragraphs[1].font.size = Pt(10)
                shape.text_frame.paragraphs[0].font.bold = True
                shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
                
                # Recursively process subordinates (if any)
                new_target_id = subordinate['Employee ID']
                if new_target_id in branch:
                    
                    # call
                    subtree_height, right_most_coor, bottom_most_coor = self.deep_search(slide, 
                        {"target": (new_target_id, shape)}, 
                        branch, 
                        right_most_coor=right_most_coor,
                        bottom_most_coor=bottom_most_coor
                        )
                    total_height_used += subtree_height
                else:
                    total_height_used += 1
            
            # Draw single vertical line for this level
            if node_positions:
                min_y = min(node_positions) - gap/2
                max_y = max(node_positions) + gap/2
                slide.shapes.add_connector(MSO_CONNECTOR.STRAIGHT, int(offset_x), int(min_y), int(offset_x), int(max_y))
                
                # Add small horizontal connectors from main vertical line to each node
                for y in node_positions:
                    slide.shapes.add_connector(MSO_CONNECTOR.STRAIGHT, int(offset_x), int(y), int(offset_x), int(y))
            
            return len(subordinates) + total_height_used, right_most_coor, bottom_most_coor
        
        return 0, right_most_coor, bottom_most_coor

In [ ]:

data = {
    'Employee ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'Supervisor ID': [0, 1, 1, 2, 2, 3, 4, 7, 7, 5, 5],
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Jack', 'Tim', 'Sarah', 'John'],
    'Department': ['HR', 'IT', 'IT', 'Finance', 'Finance', 'IT', 'Finance', 'Cyber', 'Cyber', 'HR', 'HR']
}


'''
data = {
    'Employee ID': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Supervisor ID': [None, 0, 1, 1, 2, 2, 4, 4, 6, 6, 7],
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Jack', 'Tim', 'Sarah', 'John'],
    'Department': ['HR', 'IT', 'IT', 'Finance', 'Finance', 'IT', 'Finance', 'Cyber', 'Cyber', 'HR', 'HR']
}

1 -> (2, 3)
2 -> (4, 5)
4 -> (6, 7)
6 -> (8, 9)
7 -> (10)

'''

df = pd.DataFrame(data)

direct_subordinate = df[df["Supervisor ID"] == 1]['Employee ID'].tolist() # insert ID
root = df[df["Employee ID"] == 1].iloc[0].to_dict()  # insert ID

second_level_branch = []
second_level_root = []

for id in direct_subordinate:
    level2_struc, level2_root = build_hierarchy(df, employee_id=id)
    second_level_branch.append(group(level2_struc))
    second_level_root.append(level2_root.iloc[0].to_dict())

In [155]:
org = Chart(root, second_level_root, second_level_branch, {})

In [156]:
prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[5])  

# Slide width
slide_width = prs.slide_width

# Root shape dimensions
shape_width = Inches(1.5)
shape_height = Inches(0.8)

left = (slide_width - shape_width) / 2  # Center horizontally
top = Inches(1)  # Leave space for title


org.add_rectangle_root(slide, left, top, shape_width, shape_height)

'''
c1 = slide.shapes.add_shape(
    MSO_SHAPE.OVAL, Inches(0.05), Inches(3), Inches(0.05), Inches(0.05)  # Centered in slide
)
c1.fill.solid()
c1.fill.fore_color.rgb = RGBColor(0, 0, 0)  # Black circle
'''

# If no subordinate
if not second_level_root:
    prs.save("top_center_shape_v2.pptx")
    print("Presentation saved as 'top_center_shape_v2.pptx'")
    sys.exit()
    
    
org.plot_middle(slide)
org.plot_second_level_branch(slide)

prs.save("top_center_shape_v2.pptx")
print("Presentation saved as 'top_center_shape_v2.pptx'")


Presentation saved as 'top_center_shape_v2.pptx'
